In [13]:
import os
import numpy as np
import pandas as pd
from sklearn import neighbors, metrics 
from sklearn.model_selection import train_test_split   

LOOKING INTO THE DATASETS:

In [14]:
#Importing the dataset
data_GTAV = pd.read_csv('FPS_grandTheftAuto5.csv') 
data_Fortnite = pd.read_csv('FPS_fortnite.csv') 
data_FarCry5 = pd.read_csv('FPS_farCry5.csv') 
data_CS = pd.read_csv('FPS_counterStrikeGlobalOffensive.csv') 
data_Battlefield4 = pd.read_csv('FPS_battlefield4.csv') 
data_PUBG = pd.read_csv('FPS_playerUnknownsBattlegrounds.csv') 
data_CODWW2 = pd.read_csv('FPS_callOfDutyWW2.csv') 

#printing the data just to check:
print(data_GTAV.head()) 
 

   CpuNumberOfCores  CpuNumberOfThreads  CpuBaseClock  CpuFrequency  CpuTDP  \
0                 8                  16           100          3600      95   
1                 8                   8           100          3600      95   
2                 6                  12           100          3700      95   
3                 4                   8           100          4200      91   
4                 6                   6           100          2900      65   

   CpuTurboClock  GpuBandwidth  GpuBaseClock  GpuBoostClock  GpuMemorySize  \
0           5000        616000          1350           1545          11000   
1           4900        616000          1350           1545          11000   
2           4700        616000          1350           1545          11000   
3           4500        616000          1350           1545          11000   
4           4100        616000          1350           1545          11000   

     FPS  
0  135.2  
1  134.0  
2  123.8  
3  120.2  
4

In [17]:
#Seperating the data in features and label
X_GTAV = data_GTAV.drop(columns = ['FPS']).values  
y_GTAV = data_GTAV['FPS'].values 

#Printing into arrays:
print("Features(X):\n", X_GTAV)
print("\nLabel(y):\n", y_GTAV)

Features(X):
 [[    8    16   100 ...  1350  1545 11000]
 [    8     8   100 ...  1350  1545 11000]
 [    6    12   100 ...  1350  1545 11000]
 ...
 [    6    12   100 ...  1168  1244  4000]
 [    6    12   100 ...  1168  1244  4000]
 [    8    16   100 ...  1168  1244  4000]]

Label(y):
 [135.2 134.  123.8 ...  40.3  39.8  38.6]


DOING THE SAME STUFF FOR ALL THE OTHER DATASETS

In [19]:
X_Fortnite = data_Fortnite.drop(columns = ['FPS']).values  
y_Fortnite = data_Fortnite['FPS'].values 

X_FarCry5 = data_FarCry5.drop(columns = ['FPS']).values  
y_FarCry5 = data_FarCry5['FPS'].values 

X_CS = data_CS.drop(columns = ['FPS']).values  
y_CS = data_CS['FPS'].values 

X_Battlefield4 = data_Battlefield4.drop(columns = ['FPS']).values  
y_Battlefield4 = data_Battlefield4['FPS'].values 

X_PUBG = data_PUBG.drop(columns = ['FPS']).values  
y_PUBG = data_PUBG['FPS'].values 

X_CODWW2 = data_CODWW2.drop(columns = ['FPS']).values  
y_CODWW2 = data_CODWW2['FPS'].values 
